# 导入工具包

In [31]:
import random
import numpy as np
import pandas as pd
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense

In [1]:
!pip install bert4keras -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


In [14]:
!pip install tensorflow-gpu==1.14.0 -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.14.0 (from versions: 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.2.2, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.3.2, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0, 2.4.1, 2.5.0rc0, 2.5.0rc1, 2.5.0rc2, 2.5.0rc3, 2.5.0)
ERROR: No matching distribution found for tensorflow-gpu==1.14.0


In [18]:
!pip install tensorflow-gpu==2.2.0 -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


# 设置参数

In [32]:
set_gelu('tanh')  # 切换gelu版本

In [33]:
# num_classes = 2
# maxlen = 128
# batch_size = 32
# config_path = '../model/albert_small_zh_google/albert_config_small_google.json'
# checkpoint_path = '../model/albert_small_zh_google/albert_model.ckpt'
# dict_path = '../model/albert_small_zh_google/vocab.txt'

# # 建立分词器
# tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [34]:
num_classes = 2
maxlen = 128
batch_size = 32
config_path = '../model/chinese_wwm_ext_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../model/chinese_wwm_ext_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../model/chinese_wwm_ext_L-12_H-768_A-12/vocab.txt'

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 定义模型

In [35]:
# # 加载预训练模型
# bert = build_transformer_model(
#     config_path=config_path,
#     checkpoint_path=checkpoint_path,
#     model='albert',
#     return_keras_model=False,
# )
# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='roberta',
    return_keras_model=False,
)

In [36]:
output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [37]:
# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。
# AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(1e-5),  # 用足够小的学习率
#     optimizer=AdamLR(learning_rate=1e-4, lr_schedule={
#         1000: 1,
#         2000: 0.1
#     }),
    metrics=['accuracy'],
)

# 生成数据

In [38]:
def load_data(valid_rate=0.3):
    train_file = "../data/train.csv"
    test_file = "../data/test.csv"
    
    df_train_data = pd.read_csv("../data/train.csv")
    df_test_data = pd.read_csv("../data/test.csv")
    
    train_data, valid_data, test_data = [], [], []
    
    for row_i, data in df_train_data.iterrows():
        id, level_1, level_2, level_3, level_4, content, label = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), label
        
        if random.random() > valid_rate:
            train_data.append( (id, text, int(label)) )
        else:
            valid_data.append( (id, text, int(label)) )
            
    for row_i, data in df_test_data.iterrows():
        id, level_1, level_2, level_3, level_4, content = data
        
        id, text, label = id, str(level_1) + '\t' + str(level_2) + '\t' + \
        str(level_3) + '\t' + str(level_4) + '\t' + str(content), 0
        
        test_data.append( (id, text, int(label)) )
    return train_data, valid_data, test_data

In [39]:
train_data, valid_data, test_data = load_data(valid_rate=0.3)

In [40]:
valid_data.

[(2,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t6、重点工种人员以及其他员工消防知识的掌握情况；\t消防知识要加强',
  0),
 (13,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t1、消防安全重点单位应当进行每日防火巡查，并确定巡查的人员、内容、部位和频次。其他单位可以根据需要组织防火巡查。巡查的内容应当包括：1、用火、用电有无违章情况；\t三楼车间天花板脱落，找人维修',
  0),
 (18, '纯办公场所（基础）—2016版\t（一）基础管理\t2、应急管理\t1、定期进行应急演练，有演练记录。\t未进行消防演练', 0),
 (21,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t2、安全疏散通道、疏散指示标志、应急照明和安全出口情况；\t巡查中发现紧急疏散指示灯故障，已经更换',
  0),
 (22,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t疏散指示灯故障',
  0),
 (26,
  '商贸服务教文卫类（现场）—2016版\t（二）电气安全\t3、电气线路及电源插头插座\t3、电源插座、电源插头应按规定正确接线。\t电源插头应按规定正确接线',
  1),
 (29,
  '商贸服务教文卫类（现场）—2016版\t（一）消防检查\t1、防火巡查\t4、常闭式防火门是否处于关闭状态，防火卷帘下是否堆放物品影响使用。\t一楼四号常闭式防火门，经常被杂物堆放，责令清洁部，清理干净保持防火门关闭状态',
  0),
 (35, '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t5、用火、用电有无违章情况；\t没有安装防爆灯，已整改。', 0),
 (36, '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t4、灭火器材配置及有效情况；\t有一个灭火器压力不足', 0),
 (38,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t疏散通道堆积杂物，已清理',
  0),
 (40,
  '商

In [41]:
len(train_data)

8345

In [42]:
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (id, text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

In [43]:
# 转换数据集
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)

In [44]:
valid_data

[(2,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t6、重点工种人员以及其他员工消防知识的掌握情况；\t消防知识要加强',
  0),
 (13,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t1、消防安全重点单位应当进行每日防火巡查，并确定巡查的人员、内容、部位和频次。其他单位可以根据需要组织防火巡查。巡查的内容应当包括：1、用火、用电有无违章情况；\t三楼车间天花板脱落，找人维修',
  0),
 (18, '纯办公场所（基础）—2016版\t（一）基础管理\t2、应急管理\t1、定期进行应急演练，有演练记录。\t未进行消防演练', 0),
 (21,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t2、安全疏散通道、疏散指示标志、应急照明和安全出口情况；\t巡查中发现紧急疏散指示灯故障，已经更换',
  0),
 (22,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t疏散指示灯故障',
  0),
 (26,
  '商贸服务教文卫类（现场）—2016版\t（二）电气安全\t3、电气线路及电源插头插座\t3、电源插座、电源插头应按规定正确接线。\t电源插头应按规定正确接线',
  1),
 (29,
  '商贸服务教文卫类（现场）—2016版\t（一）消防检查\t1、防火巡查\t4、常闭式防火门是否处于关闭状态，防火卷帘下是否堆放物品影响使用。\t一楼四号常闭式防火门，经常被杂物堆放，责令清洁部，清理干净保持防火门关闭状态',
  0),
 (35, '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t5、用火、用电有无违章情况；\t没有安装防爆灯，已整改。', 0),
 (36, '工业/危化品类（现场）—2016版\t（一）消防检查\t2、防火检查\t4、灭火器材配置及有效情况；\t有一个灭火器压力不足', 0),
 (38,
  '工业/危化品类（现场）—2016版\t（一）消防检查\t1、防火巡查\t2、安全出口、疏散通道是否畅通，安全疏散指示标志、应急照明是否完好；\t疏散通道堆积杂物，已清理',
  0),
 (40,
  '商

# 验证和预测

In [45]:
def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total

In [46]:
class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
#         test_acc = evaluate(test_generator)
#         print(
#             u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
#             (val_acc, self.best_val_acc, test_acc)
#         )
        test_acc = evaluate(valid_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
            (val_acc, self.best_val_acc, test_acc)
        )

In [47]:
def data_pred(test_data):
    id_ids, y_pred_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        token_ids = sequence_padding([token_ids])
        segment_ids = sequence_padding([segment_ids])
        y_pred = model.predict([token_ids, segment_ids]).argmax(axis=1)
        id_ids.append(id)
        y_pred_ids.append(y_pred)
    return id_ids, y_pred_ids

In [48]:
def data_pred2(test_data):
    id_ids, y_pred_ids = [], []
    batch_token_ids, batch_segment_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        batch_token_ids.append(token_ids)
        batch_segment_ids.append(batch_segment_ids)
        id_ids.append(id)
    token_ids = sequence_padding(batch_token_ids)
    segment_ids = sequence_padding(batch_segment_ids)
    y_pred_ids = model.predict([token_ids, segment_ids]).argmax(axis=1)
    return id_ids, y_pred_ids

In [49]:
## 修改代码

In [50]:
def data_pred3(test_data):
    id_ids, y_pred_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        token_ids = sequence_padding([token_ids])
        segment_ids = sequence_padding([segment_ids])
        y_pred = int(model.predict([token_ids, segment_ids]).argmax(axis=1)[0])
        id_ids.append(id)
        y_pred_ids.append(y_pred)
    return id_ids, y_pred_ids

In [51]:
def data_pred4(test_data):
    id_ids, y_pred_ids = [], []
    batch_token_ids, batch_segment_ids = [], []
    for id, text, label in test_data:
        token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
        batch_token_ids.append(token_ids)
        batch_segment_ids.append(batch_segment_ids)
        id_ids.append(id)
    token_ids = sequence_padding(batch_token_ids)
    segment_ids = sequence_padding(batch_segment_ids)
    y_pred_ids = model.predict([token_ids, segment_ids]).argmax(axis=1)[:,0]
    return id_ids, list(y_pred_ids)

# 训练和验证

In [52]:
evaluator = Evaluator()

In [53]:
model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=10,
        callbacks=[evaluator]
    )

/root/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1/10
261/261 [==============================] - 2122s 8s/step - loss: 0.1280 - accuracy: 0.9542
val_acc: 0.97866, best_val_acc: 0.97866, test_acc: 0.97866

Epoch 2/10
261/261 [==============================] - 2093s 8s/step - loss: 0.0523 - accuracy: 0.9818
val_acc: 0.98057, best_val_acc: 0.98057, test_acc: 0.98057

Epoch 3/10
261/261 [==============================] - 2107s 8s/step - loss: 0.0351 - accuracy: 0.9884
val_acc: 0.97784, best_val_acc: 0.98057, test_acc: 0.97784

Epoch 4/10
261/261 [==============================] - 2105s 8s/step - loss: 0.0245 - accuracy: 0.9915
val_acc: 0.97975, best_val_acc: 0.98057, test_acc: 0.97975

Epoch 5/10
261/261 [==============================] - 2103s 8s/step - loss: 0.0150 - accuracy: 0.9951
val_acc: 0.97893, best_val_acc: 0.98057, test_acc: 0.97893

Epoch 6/10
261/261 [==============================] - 2117s 8s/step - loss: 0.0109 - accuracy: 0.9969
val_acc: 0.98057, best_val_acc: 0.98057, test_acc: 0.98057

Epoch 7/10
261/261 [========

In [54]:
model.load_weights('best_model.weights')
# print(u'final test acc: %05f\n' % (evaluate(test_generator)))
print(u'final test acc: %05f\n' % (evaluate(valid_generator)))

final test acc: 0.980575



In [55]:
print(u'final test acc: %05f\n' % (evaluate(train_generator)))

final test acc: 0.991612



# 模型预测保存结果

In [56]:
id_ids, y_pred_ids = data_pred3(test_data)

In [57]:
df_save = pd.DataFrame()
df_save['id'] = id_ids
df_save['label'] = y_pred_ids

In [58]:
df_save.to_csv('result.csv')